In [1]:
import re
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk.tag import UnigramTagger #別途要否確認

In [3]:
#stop wordsの読込
#後日ここにユーザー独自のstop wordsを加えるようにする
stop_words = stopwords.words('english')

#tokenから、定義したstop_wordsとハイパーリンク等の除去、pos_tagの付与後、トークンを正規化
#　返すのは(token, pos_tag)
# lemmatizerの引数として使うpos_tagは読み込んだ時点のもの
#（lemmatize後のtokenでpos_tagを取得すると、結果は異なる可能性）

def remove_noise(tokens, stop_words=()):
    '''Remove @ mentions, hyperlinks, punctuation, and stop words'''

    clean_tokens = []

    lemmatizer = WordNetLemmatizer()

    for token, tag in pos_tag(tokens):
        # Remove Hyperlinks
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|' \
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', token)
        # Remove handles
        token = re.sub("(@[A-Za-z0-9_]+)", "", token)

        # Lemmatizerの仕様に合わせ、tokenのposタグを変換
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        # Normalize sentence
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            # Get lowercase
            clean_tokens.append((token, tag))  # (token, pos_tag) のタプルをリストに追加

    return clean_tokens


In [4]:
text = "This is a sample sentence for text mining in Python provided by Panasonic Holdings ."

# テキストをトークン化
tokens = word_tokenize(text)
# 品詞タグ付け
pos_tags = pos_tag(tokens)

print(pos_tags)


[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sample', 'JJ'), ('sentence', 'NN'), ('for', 'IN'), ('text', 'NN'), ('mining', 'NN'), ('in', 'IN'), ('Python', 'NNP'), ('provided', 'VBN'), ('by', 'IN'), ('Panasonic', 'NNP'), ('Holdings', 'NNPS'), ('.', '.')]


In [5]:
cleaned_pos_tag = remove_noise(tokens,stop_words)

print(cleaned_pos_tag)


[('sample', 'JJ'), ('sentence', 'NN'), ('text', 'NN'), ('mining', 'NN'), ('Python', 'NNP'), ('provide', 'VBN'), ('Panasonic', 'NNP'), ('Holdings', 'NNPS')]


In [9]:
# 指定された品詞を持つtoken, pos_tagを抽出
#以下は名詞のみを抽出するもの
specified_pos_tag =  [(token, tag) for token, tag in cleaned_pos_tag if tag.startswith('NN')]

print(specified_pos_tag)

[('sentence', 'NN'), ('text', 'NN'), ('mining', 'NN'), ('Python', 'NNP'), ('Panasonic', 'NNP'), ('Holdings', 'NNPS')]


In [7]:
#tokenのみのリスト

specified_tokens = [token for token, _ in specified_pos_tag]
print(specified_tokens)


['sentence', 'text', 'mining', 'Python', 'Panasonic', 'Holdings']


In [8]:



# カスタム辞書を読み込む関数
def load_custom_dict(file_path):
    custom_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            word, pos_tag = line.strip().split()
            custom_dict[word] = pos_tag
    return custom_dict

# カスタム辞書ファイルのパス
custom_dict_file = "userdic/custom_dict.txt"

# カスタム辞書を読み込む
custom_vocab = load_custom_dict(custom_dict_file)

# NLTKの標準品詞タガーを取得
#default_tagger = nltk.data.load('taggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle')
#default_tagger = nltk.data.load('/root/nltk_data/taggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle')


# カスタムタガーを作成し、標準タガーをバックアップとして指定
#custom_tagger = UnigramTagger(model=custom_vocab, backoff=default_tagger)

# テキストの定義
text = "This is a sample sentence for text mining in Python provided by Panasonic Holdings."


# テキストのトークン化と品詞タギング
tokens = nltk.word_tokenize(text)
pos_tags = nltk.pos_tag(tokens, tagset='universal', lexicon=custom_vocab)
#pos_tags = custom_tagger.tag(tokens)
print(pos_tags)

# 指定された品詞を持つトークンを抽出
specified_pos_tokens = [token for token, pos_tag in pos_tags if pos_tag in ['NN', 'NNP']]
print(specified_pos_tokens)

TypeError: pos_tag() got an unexpected keyword argument 'lexicon'

In [ ]:
# テキストをリスト単位で取り込み、リスト単位で形態素解析

import MeCab
import fitz
import re

doc = fitz.open("data/pana_ar2023j_a4.pdf") # ドキュメントを開く

#ページ単位でテキストをリストに格納
#page_textにはインデックスは付与されていない
page_text=[]
for page in doc:
    text = page.get_text() # プレーンテキストを取得
    #if text != '' :
        #無駄な改行を削除の上で'。'で改行、一文の範囲を明確にする
    text = re.sub(r'。','。\n', re.sub(r'\n','', text))
        #リストに格納
    page_text.append(text)
#print(page_text)

In [ ]:
def load_exclusion_list(file_path):
    exclusion_list = []
    try:
        with open(file_path, 'r') as file:
            exclusion_list = [line.strip() for line in file]
    except FileNotFoundError:
        pass
    return exclusion_list

def noise_eliminator(text):
#形態素解析の前に、無駄な記号やヘッダ・フッタ等の文言をテキストから除外

    replaced_text = text

    #exclusion_list処理前に処理する必要のあるもの
    #replaced_text = re.sub(r'[▶]', '', replaced_text)

    exclusion_file = "userdic/exclusion_list.txt"  # 除外する項目を記載したテキストファイル
    exclusion_list = load_exclusion_list(exclusion_file)

    for pattern in exclusion_list:
        replaced_text = re.sub(pattern, ' ', replaced_text)
    
    replaced_text = re.sub(r'P\d+', ' ', replaced_text) 
        
    replaced_text = re.sub(r'[■□▪▫▲△▶▷▸▹▼▽◆◇●〇]', '。\n', replaced_text)#文頭のこれら記号は箇条書きとして一文とみなす扱い
    #replaced_text = re.sub(r'[〇●◇◆□■▶△▲▽▼▫▪▹▶▸]', '', replaced_text)#上記以外は除去
    
    replaced_text = re.sub(r'[【】（）()\[\]]', ' ', replaced_text)
    replaced_text = re.sub(r'[\d\-]+年度末', '', replaced_text)  # 年度末の除去
    replaced_text = re.sub(r'[\d\-]+年度', '', replaced_text)  # 年度の除去
    replaced_text = re.sub(r'\d+[年月日]', '', replaced_text)  # 年月日の除去
    replaced_text = re.sub(r'\d+回', '', replaced_text)  # \d回（取締役の任命回数など）の除去

    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?://[\w/;%#\$\&\?\(\)~\.=\+\-]+', '', replaced_text)  # リンクの除去
    #replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #　数字を除去
    #replaced_text = text.lower() #　すべて小文字に変換

    return replaced_text


In [ ]:
#形態素解析関数（入力：テキスト、出力：リスト）

def mecab_tokenizer(text):

    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
   
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    #[:-2]で文末の'EOS',''の取り込みを抑制
    #print(parsed_lines)
    
    #形態素の取得（リスト形式）
    #原形を取得
    token_list = [l.split("\t")[2] for l in parsed_lines]
    #表層形を確認したい場合
    #token_list = [l.split('\t')[0] for l in parsed_lines]
    #print(token_list)
    
    #品詞区分による絞り込み
    #--------------------------------------------
    #Chasenの品詞区分を個別に指定する場合
    
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #Chasenの品詞区分の大項目のみで絞り込む場合
    #品詞区分の取得（リスト形式）
    #pos = [l.split('\t')[3].split("-")[0]  for l in parsed_lines]
    
    #絞り込む品詞区分の定義
    #target_pos = ['名詞']
    
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    
    #stopwordsの指定
    stopwords = []

    with open("userdic/Japanese.txt","r") as f:
        stopwords1 = f.read().split("\n")
   #stopwordsを直接反映したい場合は以下のリストに記載
    stopwords2 =["以下","ため","当社","当行","場合","影響","可能性",
            "状況","グループ","こと","平成","令和","年","月","期","当","他",
            "等","お","これ","%","以上","もの","株式会社",
            "もの","とも","ある","よる","的","化","お呼び",
            "CEO","会長","社長","副社長","専務","役員","常務","代表社員"
            "代表取締役会長","代表取締役社長","代表取締役","常務取締役","社外取締役","取締役会長","取締役社長",
            "代表執行役員","専務執行役員","常務執行役員","執行役員",
            "取締役","取締役会","監査役","監査役会","議長","所長",
            "担当","100％","100%","株","データ","男性","チーフ","オフィサー","CFO",
            "入社","入所","就任","選任","指名","任命","解任","諮問","答申","ご参照","全体",
            "*"]
    stopwords.extend(stopwords1)
    stopwords.extend(stopwords2)
    # 元の順序を保持しつつ重複を除去（# Python 3.7以降）
    from collections import OrderedDict
    stopwords = list(OrderedDict.fromkeys(stopwords))
    
    # stopwordsの除去
    token_list = [t for t in token_list if t  not in stopwords]
    
    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]

    # アルファベット1文字のみの形態素を除外
    alphabet_re = re.compile("^[a-zA-Z]$")
    token_list = [t for t in token_list if not alphabet_re.match(t)]

    return token_list

In [ ]:
#ページインデックス付きで、形態素解析結果をページ単位で格納
tokenized_page_text = [(index, mecab_tokenizer(noise_eliminator(item)))for index, item in enumerate(page_text)]
#print(page_text[1])
#print()
#print(tokenized_page_text)

#ページインデックスのないもの
#tokenized_page_text_noindex = [sublist for _, sublist in tokenized_page_text]
#print (tokenized_page_text_noindex)
#ページ単位の形態素解析結果を、一つの要素にマージしたもの
#merged_tokenized_list = [item for _, sublist in tokenized_page_text for item in sublist]
#print (merged_tokenized_list)

In [ ]:
#形態素を別途用意した対比リストと比較し、対比リスト内の単語が含まれる要素のみを残す
# 対比リストを読込
with open("data/CGC_Term.txt", "r") as Ref_term:
        Ref_list = list(set(Ref_term.read().split("\n")))
#print(Ref_list)

#対比リストの単語が含まれるページの形態素リスト
Extracted_tokenized_page_text = [(index, sublist) for index, sublist in tokenized_page_text if any(item in Ref_list for item in sublist)]
#該当ページのインデックスリスト
Extracted_tokenized_page_index = [index for index, sublist in tokenized_page_text if any(item in Ref_list for item in sublist)]
#形態素解析前のpage_textの該当ページ部分
Extracted_page_text = [page_text[i] for i in Extracted_tokenized_page_index]

print(Extracted_tokenized_page_index)
#print(Extracted_tokenized_page_text)

In [ ]:
#マニュアルで抽出ページを指定
#man_page_index =[2, 3, 4, 6, 12, 13, 15, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 36, 41, 42, 43, 45, 46, 48, 49, 50, 51, 53, 55]
man_page_index = [2, 3, 4, 6, 12, 13, 15, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 36, 42, 49, 50, 51, 53, 55]
#man_page_index = [2, 5, 14, 19, 22, 25, 40, 48, 49, 52]

#抽出ページ単位の形態素リスト
Extracted_tokenized_page_text = [(index, sublist) for index, sublist in tokenized_page_text if index in man_page_index]
#該当ページのインデックスリスト
Extracted_tokenized_page_index = [index for index, sublist in tokenized_page_text if index in man_page_index]
#形態素解析前の抽出ページ単位テキスト
Extracted_page_text = [page_text[i] for i in man_page_index]
#特定のページのみを直接指定するなら、
#Extracted_page_text = [page_text[5]]

#print(Extracted_tokenized_page_index)
#print(Extracted_tokenized_page_text[2])
#print(Extracted_page_text[2])

In [ ]:
#抽出されたページのサブリストを一つのリストにまとめる
Merged_Extracted_tokenized_list = [item for _, sublist in Extracted_tokenized_page_text for item in sublist]
#print(merged_Extracted_tokenized_list)

In [ ]:
import collections
from collections import Counter
import pandas as pd
import itertools
import datetime

#辞書形式で単語をカウント
counter = Counter(Merged_Extracted_tokenized_list)
#for word, count in counter.most_common(500):
#    print('%s : %s' % (word, count))

# 単語、件数をDataFrameに格納
count_df = pd.DataFrame(list(counter.items()), columns=['単語', '件数'])
# DataFrameを件数でソート
count_df = count_df.sort_values(by='件数', ascending=False)
#ファイル名を指定

# 現在の日付と時刻を取得
now = datetime.datetime.now()
# 年月日と時刻の文字列を生成
date_time_string = now.strftime("%Y%m%d_%H%M%S")
# ファイル名を生成
file_name = f"data/Word_list_{date_time_string}.csv"
# 結果をCSVファイルに出力
count_df.to_csv(file_name, encoding="utf_8_sig", index=False)

In [ ]:
#以降、共起分析

#Extracted_page_text（抽出ページ単位のリスト）を、要素一つのリストに集約
Merged_Extracted_page_text = "\n\n".join(Extracted_page_text)
#print(noise_eliminator(Merged_Extracted_page_text))

In [ ]:
#集約したテキストを「。」で区切り、形態素解析（共起分析用）
sentences = [mecab_tokenizer(sentence) for sentence in re.split("。", noise_eliminator(Merged_Extracted_page_text))]
#sentences = [mecab_tokenizer(sentence) for sentence in re.split("。|\n", noise_eliminator(Merged_Extracted_page_text))]

#print(sentences)

In [ ]:
#各文中の、形態素組み合わせを作る
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
#print(sentences_combs[0])

#組み合わせた2つの形態素の並びをソート
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
#print(words_combs[0][:30])

target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
#print(target_combs[:30])

ct = collections.Counter(target_combs)
#print(ct)

#import pandas as pd
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
df.head(30)

# 現在の日付と時刻を取得
#now = datetime.datetime.now()
# 年月日と時刻の文字列を生成
#date_time_string = now.strftime("%Y%m%d_%H%M%S")
# ファイル名を生成
# date_timeはWordlistと同じにしておく
file_name_comb = f"data/co_count_result_{date_time_string}.csv"
df.to_csv(file_name_comb, encoding="utf_8_sig", index=False)

In [ ]:
#ネットワーク分析の下準備
import networkx as nx
from networkx.algorithms.community import girvan_newman
from pyvis.network import Network
import random
#import matplotlib
#import matplotlib.pyplot as plt
#plt.rcParams['font.family'] = 'IPAexGothic'  # 使用するフォントを指定（例：IPAexGothic）
#import japanize_matplotlib
#japanize_matplotlib.japanize()


In [ ]:
#########################################################
# 分析対象とする共起単語の組み合わせ数（ノード数）を指定
analyzed_nodes=1000
limited_df=df.head(analyzed_nodes)
#########################################################

In [ ]:
# DataFrameからネットワークを作成
G = nx.from_pandas_edgelist(limited_df, '1番目', '2番目', ['count'])
#print(G)

# 各ノードの中心性を計算
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
#katz_centrality = nx.katz_centrality(G)

# Girvan-Newmanアルゴリズムでコミュニティに分割
comp = girvan_newman(G)
communities = tuple(sorted(c) for c in next(comp))

# 各ノードがどのコミュニティに属するかを記録
community_map = {}
for i, community in enumerate(communities):
    for node in community:
        community_map[node] = i

# 中心性を新しいデータフレームに格納
centrality_df = pd.DataFrame({
    'Node': list(G.nodes()),
    'Degree Centrality': [degree_centrality[node] for node in G.nodes()],
    'Betweenness Centrality': [betweenness_centrality[node] for node in G.nodes()],
    'Closeness Centrality': [closeness_centrality[node] for node in G.nodes()],
    'Eigenvector Centrality': [eigenvector_centrality[node] for node in G.nodes()],
#    'Katz Centrality': [katz_centrality[node] for node in G.nodes()],
    'Community': [community_map[node] for node in G.nodes()]  # コミュニティ情報を追加
})
#print(centrality_df)

file_name_comb = f"data/Centrality_{date_time_string}.csv"
centrality_df.to_csv(file_name_comb, encoding="utf_8_sig", index=False)

In [ ]:
#ネットワーク図の描画
#ここではpyvisを使用

class ColoredObject:
    def __init__(self, color):
        self.color = color
# 12種類の色を用意
colors = ["#CBCCCD", "red", "blue", "green", "#A2CEF6", "magenta", "brown",  "yellow", "orange", "purple", "#EDABE5", "black" ]
# 色の順番を追跡するためのインデックス
color_index = 0
# 色を順番に取得する関数
def choose_color():
    global color_index
    color = colors[color_index]
    color_index = (color_index + 1) % len(colors)  # インデックスを更新し、13番目の場合は最初の色に戻る
    return color

def kyoki_word_network(df):

    #Girvan-Newmanアルゴリズムによるコミュニティ分割（networkxの機能）
    G = nx.from_pandas_edgelist(df, '1番目', '2番目', ['count'])
    comp = girvan_newman(G)
    communities = tuple(sorted(c) for c in next(comp))

    #各コミュニティに色を割り当てる
    color_map = {}
    for i, community in enumerate(communities):
        color = choose_color() #"#{:06x}".format(random.randint(0, 0xFFFFFF))  # ランダムな色を生成
        for node in community:
            color_map[node] = color
    
    
    got_net = Network(height="1000px", width="95%", bgcolor="#FFFFFF", font_color="black", notebook=True)

    got_net.force_atlas_2based()
    got_data = df

    sources = got_data['1番目']
    targets = got_data['2番目']
    weights = got_data['count']

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]
        got_net.add_node(src, src, title=src, color=color_map.get(src, None))
        got_net.add_node(dst, dst, title=dst, color=color_map.get(dst, None))
        got_net.add_edge(src, dst, value=w)

    # コミュニティごとにノードの色を設定
    for node, color in color_map.items():
        got_net.add_node(node, color=color)

    neighbor_map = got_net.get_adj_list()

    for node in got_net.nodes:
        node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
        node["value"] = len(neighbor_map[node["id"]])

    got_net.show_buttons(filter_=['physics'])
    return got_net

In [ ]:
got_net = kyoki_word_network(limited_df)
got_net.show("kyoki_"+str(analyzed_nodes)+".html")

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

custom_dict = {"word1": "NN", "word2": "VB"}

sentence = "Your sentence here"
tokens = word_tokenize(sentence)
tagged_tokens = nltk.pos_tag(tokens, tagset='universal', lexicon=custom_dict)


In [ ]:
import nltk
from nltk.tag import PerceptronTagger

# ユーザー辞書を読み込む関数
def load_custom_dict(file_path):
    custom_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            word, pos_tag = line.strip().split()
            custom_dict[word] = pos_tag
    return custom_dict

# カスタム辞書ファイルのパス
custom_dict_file = "userdic/custom_dict.txt"

# カスタム辞書を読み込む
custom_vocab = load_custom_dict(custom_dict_file)

# PerceptronTaggerの初期化
tagger = PerceptronTagger(load=False)

# ユーザー辞書をタガーに追加
tagger.classes = {}
tagger.model.classes = tagger.classes
tagger.tagdict.update(custom_vocab)

# テキストの定義
text = "This is a sample sentence for text mining in Pytho provided by Panasonic."

# テキストのトークン化と品詞タギング
tokens = nltk.word_tokenize(text)
pos_tags = tagger.tag(tokens)
print(pos_tags)
